# Part 1. 활성화 함수
# 1. 활성화 함수와 Sigmoid 함수
### (1) 활성화 함수
 - 활성화 함수 : 입력 신호의 총합을 출력 신호로 변환하는 함수이다.
 - 활성화 함수로 Sigmoid, ReLU 등이 있다.
 
 <br>
 
### (2) Sigmoid 함수
 - x = torch.nn.sigmoid(x)
 - Sigmoid 함수 : S자 모양의 함수로, 넓은 범위의 숫자를 0~1 사이의 숫자로 출력값을 바꿔준다.
 - Sigmoid의 특징 : Sigmoid 함수는 S자 모양인데, 0과 1 사이는 기울기가 완만하지만, 0이나 1에 가까워 질 수록 기울기가 급격하게 0에 수렴하게 생겼다.
 - 학습 과정 : Input -> Layer -> Sigmoid -> Output -> Cost 계산(GT와 Output의 차이) -> Backpropagation

# 2. Sigmoid 함수의 문제점
### (1) Vanishing Gradient
 - Sigmoid의 문제점은 Backpropagation 과정에서 발생한다. 
 - Sigmoid의 0이나 1에 가까운 부분의 기울기는 0.xx처럼 굉장히 낮다. 
 - Backpropagation을 할 때 기울기를 곱하며 계산하게 되는데, 이때 0.xx가 곱해지면 기울기가 점차 소멸되게 된다.
 - 여러 레이어들을 쌓을 때 활성화함수로 Sigmoid를 이용하면, Backpropagation 도중 마지막단 레이어들은 기울기(Gradient)가 잘 전달 되겠지만, 앞단의 레이어는 기울기를 거의 전파받지 못한다.

<br>

### (2) 해결법
 - ReLU!

# 3. ReLU 함수
### (1) ReLU
 - x = torch.nn.relu(x)
 - f(x) = max(0, x)
 - ReLU 함수는 함수에 x가 들어왔을 때, x가 0보다 크면 원래 x값 그대로 출력하고, 0보다 작으면 0으로 출력한다.
 - 음수가 들어오면 기울기(Gradient)가 0이 되고, 양수가 들어오면 기울기가 1이 된다. (원래 값 그대로 이기 때문에 기울기 1)

<br>

### (2) ReLU와 Vanishing Gradient
 - 기울기가 0 또는 1이기 때문에 Vanishing Gradient 문제가 없다!

<br>

### (3) ReLU의 문제점
 - 음수가 들어오면 기울기가 아예 사라진다.
 - but 대부분 잘 동작함!

# 4. 기타 활성화 함수
 - torch.nn.tanh(x)
 - torch.nn.leaky_relu(x, 0.01) : relu에서 0값 내보내는 걸 약간 변형

# Part 2. Optimizer
# 1. PyTorch의 다양한 Optimizer들
### (1) PyTorch의 Optimizer
 - torch.optim에는 다양한 optimization algorithms들이 있다.
 - Optimizer는 그라 디센드를 수행하는 애들임!
 - 시간이 있다면 같은 모델에 다양한 Optimizer를 적용해서 결과를 비교해보는 것도 좋겠다.

<br>

### (2) 다양한 Optimizer들
 - torch.optim.SGD
 - torch.optim.Adadelta
 - torch.optim.Adagrad
 - torch.optim.Adam
 - torch.optim.SparseAdam
 - torch.optim.Adamax
 - torch.optim.ASGD
 - torch.optim.LBFGS
 - torch.optim.RMSprop
 - torch.optim.Rprop
 
<br>

### (3) 각 Optimizer들의 전략
 - SGD : 조금만 보고 빨리 판단해서 깎아 내려가자
 - Momentum : 아까 내려 오던 관성 방향대로 깎아 내려가자
 - Adam : 방향도 스텝사이즈도 적절하게 깎아 내려가자

# Part 3. MNIST 분류
# 1. MNIST softmax로 분류 - 기존에 배운것에서 Optimizer만 Adam으로 바꿔보기!

In [1]:
import torch
import torchvision.datasets as dsets # torchvision의 datasets를 불러와서 dsets로 이용
import torchvision.transforms as transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu' # cuda가 사용 가능한 상황이면 cuda를 쓰고, 아니면 CPU를 사용

# 파라미터들
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# 데이터 불러오기
mnist_train = dsets.MNIST(root = "MNIST_data/", train = True, transform = transforms.ToTensor(), download = True) # MNIST 함수를 이용함. root로 경로 지정, train으로 train set 불러올지 설정, transform으로 MNIST 일반 이미지를 PyTorch가 이미지 인식하는 방법대로 읽어오게 설정. download로 root에 MNIST 데이터가 없으면 다운을 받아오게 설정.
mnist_test = dsets.MNIST(root = "MNIST_data/", train = False, transform = transforms.ToTensor(), download=True)
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, batch_size = batch_size, shuffle=True, drop_last = True) # 데이터를 실제로 사용하기 위해서 DataLoader를 이용한다. DataLoader로 어떤 데이터를 로드 할 지 설정, batch_size로 총 60,000개의 데이터를 몇개 씩 잘라서 가져올 지 설정, shuffle로 데이터를 랜덤 순서로 불러올지 설정, drop_last로 batch_size로 짤라서 불러 올 때 맨 마지막에 남은 찌끄러기들을 사용 안 할지 설정

# Fully-connected 레이어 하나!
linear = torch.nn.Linear(784, 10, bias = True).to(device) # Linear 모델로, 784(28 * 28)개의 인풋을 받아 최종적으로 10(0 ~ 9)개의 클래스로 분류함

torch.nn.init.normal_(linear.weight) # layer의 weight 초기화!

criterion = torch.nn.CrossEntropyLoss().to(device) # Loss

# 기존에 배운것에서 이 부분만 바뀜!!
optimizer = torch.optim.Adam(linear.parameters(), lr = learning_rate) # Optimizer를 Adam으로 설정

# 학습
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = len(data_loader)
  
    for X, Y in data_loader:
        X = X.view(-1, 28*28).to(device)
        Y = Y.to(device)
    
        hypothesis = linear(X)
        cost = criterion(hypothesis, Y)
    
        optimizer.zero_grad()
        cost.backward()
        avg_cost += cost / total_batch
    
        print("Epoch: ", "%04d" % (epoch+1), "cost =", "{:.9f}".format(avg_cost)) # Accuracy : 89%

Epoch:  0001 cost = 0.018031320
Epoch:  0001 cost = 0.037115354
Epoch:  0001 cost = 0.057401057
Epoch:  0001 cost = 0.079386413
Epoch:  0001 cost = 0.099407256
Epoch:  0001 cost = 0.121829450
Epoch:  0001 cost = 0.139409453
Epoch:  0001 cost = 0.158599451
Epoch:  0001 cost = 0.181855559
Epoch:  0001 cost = 0.201566368
Epoch:  0001 cost = 0.223408893
Epoch:  0001 cost = 0.244842514
Epoch:  0001 cost = 0.264247030
Epoch:  0001 cost = 0.283661425
Epoch:  0001 cost = 0.305209756
Epoch:  0001 cost = 0.327677637
Epoch:  0001 cost = 0.347746283
Epoch:  0001 cost = 0.368600368
Epoch:  0001 cost = 0.388411999
Epoch:  0001 cost = 0.409676760
Epoch:  0001 cost = 0.429264039
Epoch:  0001 cost = 0.450412214
Epoch:  0001 cost = 0.472362548
Epoch:  0001 cost = 0.493454933
Epoch:  0001 cost = 0.516187429
Epoch:  0001 cost = 0.536198258
Epoch:  0001 cost = 0.558468521
Epoch:  0001 cost = 0.581327438
Epoch:  0001 cost = 0.602481067
Epoch:  0001 cost = 0.621215284
Epoch:  0001 cost = 0.643557727
Epoch:  

Epoch:  0001 cost = 5.496800900
Epoch:  0001 cost = 5.517901897
Epoch:  0001 cost = 5.535321712
Epoch:  0001 cost = 5.554198265
Epoch:  0001 cost = 5.576131344
Epoch:  0001 cost = 5.597254753
Epoch:  0001 cost = 5.618419170
Epoch:  0001 cost = 5.637471676
Epoch:  0001 cost = 5.657745361
Epoch:  0001 cost = 5.675992012
Epoch:  0001 cost = 5.697340012
Epoch:  0001 cost = 5.718756199
Epoch:  0001 cost = 5.740238190
Epoch:  0001 cost = 5.760871410
Epoch:  0001 cost = 5.780632019
Epoch:  0001 cost = 5.801890373
Epoch:  0001 cost = 5.822432518
Epoch:  0001 cost = 5.843606472
Epoch:  0001 cost = 5.865635872
Epoch:  0001 cost = 5.887135983
Epoch:  0001 cost = 5.908597946
Epoch:  0001 cost = 5.929324150
Epoch:  0001 cost = 5.949106216
Epoch:  0001 cost = 5.971108437
Epoch:  0001 cost = 5.991145611
Epoch:  0001 cost = 6.012639999
Epoch:  0001 cost = 6.033848763
Epoch:  0001 cost = 6.054262161
Epoch:  0001 cost = 6.076092243
Epoch:  0001 cost = 6.095633984
Epoch:  0001 cost = 6.116222858
Epoch:  

Epoch:  0001 cost = 10.943578720
Epoch:  0001 cost = 10.963004112
Epoch:  0001 cost = 10.983571053
Epoch:  0001 cost = 11.004794121
Epoch:  0001 cost = 11.022056580
Epoch:  0001 cost = 11.041241646
Epoch:  0001 cost = 11.061477661
Epoch:  0001 cost = 11.081830025
Epoch:  0001 cost = 11.104726791
Epoch:  0001 cost = 11.126077652
Epoch:  0001 cost = 11.147389412
Epoch:  0001 cost = 11.168613434
Epoch:  0001 cost = 11.189825058
Epoch:  0001 cost = 11.208776474
Epoch:  0001 cost = 11.229978561
Epoch:  0001 cost = 11.249977112
Epoch:  0001 cost = 11.269778252
Epoch:  0001 cost = 11.294108391
Epoch:  0001 cost = 11.316537857
Epoch:  0001 cost = 11.337024689
Epoch:  0001 cost = 11.358321190
Epoch:  0001 cost = 11.380510330
Epoch:  0001 cost = 11.400490761
Epoch:  0001 cost = 11.421257973
Epoch:  0001 cost = 11.441405296
Epoch:  0001 cost = 11.459859848
Epoch:  0001 cost = 11.479592323
Epoch:  0001 cost = 11.500202179
Epoch:  0001 cost = 11.521800041
Epoch:  0001 cost = 11.543499947
Epoch:  00

Epoch:  0002 cost = 3.979291677
Epoch:  0002 cost = 4.001075268
Epoch:  0002 cost = 4.022555351
Epoch:  0002 cost = 4.044335365
Epoch:  0002 cost = 4.065337658
Epoch:  0002 cost = 4.085564137
Epoch:  0002 cost = 4.107351780
Epoch:  0002 cost = 4.127371788
Epoch:  0002 cost = 4.145287514
Epoch:  0002 cost = 4.165349483
Epoch:  0002 cost = 4.185038567
Epoch:  0002 cost = 4.205831528
Epoch:  0002 cost = 4.226653099
Epoch:  0002 cost = 4.248134136
Epoch:  0002 cost = 4.267271996
Epoch:  0002 cost = 4.289330959
Epoch:  0002 cost = 4.310290813
Epoch:  0002 cost = 4.329493999
Epoch:  0002 cost = 4.348678589
Epoch:  0002 cost = 4.368775845
Epoch:  0002 cost = 4.388989449
Epoch:  0002 cost = 4.409636497
Epoch:  0002 cost = 4.431756496
Epoch:  0002 cost = 4.452041149
Epoch:  0002 cost = 4.474413872
Epoch:  0002 cost = 4.495230198
Epoch:  0002 cost = 4.516143799
Epoch:  0002 cost = 4.535909653
Epoch:  0002 cost = 4.558588505
Epoch:  0002 cost = 4.578945637
Epoch:  0002 cost = 4.600702763
Epoch:  

Epoch:  0002 cost = 9.361598969
Epoch:  0002 cost = 9.382065773
Epoch:  0002 cost = 9.401488304
Epoch:  0002 cost = 9.420065880
Epoch:  0002 cost = 9.441750526
Epoch:  0002 cost = 9.462766647
Epoch:  0002 cost = 9.482707977
Epoch:  0002 cost = 9.502019882
Epoch:  0002 cost = 9.522336960
Epoch:  0002 cost = 9.543174744
Epoch:  0002 cost = 9.563547134
Epoch:  0002 cost = 9.583786964
Epoch:  0002 cost = 9.606932640
Epoch:  0002 cost = 9.628248215
Epoch:  0002 cost = 9.646407127
Epoch:  0002 cost = 9.666299820
Epoch:  0002 cost = 9.686649323
Epoch:  0002 cost = 9.707448006
Epoch:  0002 cost = 9.727494240
Epoch:  0002 cost = 9.748791695
Epoch:  0002 cost = 9.770755768
Epoch:  0002 cost = 9.792757034
Epoch:  0002 cost = 9.814040184
Epoch:  0002 cost = 9.834699631
Epoch:  0002 cost = 9.854870796
Epoch:  0002 cost = 9.874440193
Epoch:  0002 cost = 9.894783974
Epoch:  0002 cost = 9.916415215
Epoch:  0002 cost = 9.937518120
Epoch:  0002 cost = 9.957554817
Epoch:  0002 cost = 9.977610588
Epoch:  

...

Epoch:  0013 cost = 10.296261787
Epoch:  0013 cost = 10.316438675
Epoch:  0013 cost = 10.336899757
Epoch:  0013 cost = 10.358063698
Epoch:  0013 cost = 10.380854607
Epoch:  0013 cost = 10.400362968
Epoch:  0013 cost = 10.420659065
Epoch:  0013 cost = 10.440496445
Epoch:  0013 cost = 10.460997581
Epoch:  0013 cost = 10.482165337
Epoch:  0013 cost = 10.504140854
Epoch:  0013 cost = 10.525875092
Epoch:  0013 cost = 10.549917221
Epoch:  0013 cost = 10.569325447
Epoch:  0013 cost = 10.589395523
Epoch:  0013 cost = 10.611872673
Epoch:  0013 cost = 10.634659767
Epoch:  0013 cost = 10.653442383
Epoch:  0013 cost = 10.675781250
Epoch:  0013 cost = 10.695300102
Epoch:  0013 cost = 10.717240334
Epoch:  0013 cost = 10.737148285
Epoch:  0013 cost = 10.758220673
Epoch:  0013 cost = 10.778247833
Epoch:  0013 cost = 10.800597191
Epoch:  0013 cost = 10.822527885
Epoch:  0013 cost = 10.843338966
Epoch:  0013 cost = 10.864325523
Epoch:  0013 cost = 10.883471489
Epoch:  0013 cost = 10.904252052
Epoch:  00

Epoch:  0014 cost = 3.199009657
Epoch:  0014 cost = 3.221192122
Epoch:  0014 cost = 3.243290901
Epoch:  0014 cost = 3.261840343
Epoch:  0014 cost = 3.283062220
Epoch:  0014 cost = 3.303442240
Epoch:  0014 cost = 3.323699236
Epoch:  0014 cost = 3.344602346
Epoch:  0014 cost = 3.362234592
Epoch:  0014 cost = 3.384066582
Epoch:  0014 cost = 3.403164387
Epoch:  0014 cost = 3.424164057
Epoch:  0014 cost = 3.445633411
Epoch:  0014 cost = 3.466926336
Epoch:  0014 cost = 3.487800837
Epoch:  0014 cost = 3.509009361
Epoch:  0014 cost = 3.530367613
Epoch:  0014 cost = 3.549360991
Epoch:  0014 cost = 3.569221258
Epoch:  0014 cost = 3.592161894
Epoch:  0014 cost = 3.614158154
Epoch:  0014 cost = 3.635029793
Epoch:  0014 cost = 3.655802011
Epoch:  0014 cost = 3.676901817
Epoch:  0014 cost = 3.697507381
Epoch:  0014 cost = 3.717996836
Epoch:  0014 cost = 3.740283966
Epoch:  0014 cost = 3.761649609
Epoch:  0014 cost = 3.782392263
Epoch:  0014 cost = 3.802516222
Epoch:  0014 cost = 3.820781708
Epoch:  

Epoch:  0014 cost = 8.792976379
Epoch:  0014 cost = 8.813718796
Epoch:  0014 cost = 8.834778786
Epoch:  0014 cost = 8.857059479
Epoch:  0014 cost = 8.875261307
Epoch:  0014 cost = 8.895494461
Epoch:  0014 cost = 8.915477753
Epoch:  0014 cost = 8.937040329
Epoch:  0014 cost = 8.957257271
Epoch:  0014 cost = 8.978201866
Epoch:  0014 cost = 8.997617722
Epoch:  0014 cost = 9.017198563
Epoch:  0014 cost = 9.037950516
Epoch:  0014 cost = 9.059952736
Epoch:  0014 cost = 9.080580711
Epoch:  0014 cost = 9.100360870
Epoch:  0014 cost = 9.121133804
Epoch:  0014 cost = 9.141323090
Epoch:  0014 cost = 9.162527084
Epoch:  0014 cost = 9.182666779
Epoch:  0014 cost = 9.204320908
Epoch:  0014 cost = 9.225112915
Epoch:  0014 cost = 9.244220734
Epoch:  0014 cost = 9.264815331
Epoch:  0014 cost = 9.286840439
Epoch:  0014 cost = 9.306627274
Epoch:  0014 cost = 9.326468468
Epoch:  0014 cost = 9.347926140
Epoch:  0014 cost = 9.367760658
Epoch:  0014 cost = 9.390305519
Epoch:  0014 cost = 9.411198616
Epoch:  

Epoch:  0015 cost = 1.900262475
Epoch:  0015 cost = 1.918607593
Epoch:  0015 cost = 1.938894391
Epoch:  0015 cost = 1.959082246
Epoch:  0015 cost = 1.979903460
Epoch:  0015 cost = 2.000563383
Epoch:  0015 cost = 2.021752357
Epoch:  0015 cost = 2.045360327
Epoch:  0015 cost = 2.064412355
Epoch:  0015 cost = 2.087720871
Epoch:  0015 cost = 2.110428333
Epoch:  0015 cost = 2.128667831
Epoch:  0015 cost = 2.147476673
Epoch:  0015 cost = 2.167325258
Epoch:  0015 cost = 2.187340975
Epoch:  0015 cost = 2.208184004
Epoch:  0015 cost = 2.228241444
Epoch:  0015 cost = 2.249341249
Epoch:  0015 cost = 2.271040440
Epoch:  0015 cost = 2.292595625
Epoch:  0015 cost = 2.313206434
Epoch:  0015 cost = 2.332668543
Epoch:  0015 cost = 2.353703260
Epoch:  0015 cost = 2.373362303
Epoch:  0015 cost = 2.394476175
Epoch:  0015 cost = 2.415321112
Epoch:  0015 cost = 2.437922001
Epoch:  0015 cost = 2.459925413
Epoch:  0015 cost = 2.479771137
Epoch:  0015 cost = 2.500365257
Epoch:  0015 cost = 2.522041321
Epoch:  

Epoch:  0015 cost = 7.436004162
Epoch:  0015 cost = 7.454596043
Epoch:  0015 cost = 7.475272655
Epoch:  0015 cost = 7.494532585
Epoch:  0015 cost = 7.514587402
Epoch:  0015 cost = 7.534752846
Epoch:  0015 cost = 7.556159019
Epoch:  0015 cost = 7.578273296
Epoch:  0015 cost = 7.599210739
Epoch:  0015 cost = 7.621241093
Epoch:  0015 cost = 7.638243675
Epoch:  0015 cost = 7.659852505
Epoch:  0015 cost = 7.682435036
Epoch:  0015 cost = 7.702914238
Epoch:  0015 cost = 7.722231388
Epoch:  0015 cost = 7.743412495
Epoch:  0015 cost = 7.763697147
Epoch:  0015 cost = 7.785799980
Epoch:  0015 cost = 7.806427956
Epoch:  0015 cost = 7.828614712
Epoch:  0015 cost = 7.850276470
Epoch:  0015 cost = 7.870493889
Epoch:  0015 cost = 7.891735077
Epoch:  0015 cost = 7.910305023
Epoch:  0015 cost = 7.931322098
Epoch:  0015 cost = 7.953588009
Epoch:  0015 cost = 7.973202705
Epoch:  0015 cost = 7.991101742
Epoch:  0015 cost = 8.012113571
Epoch:  0015 cost = 8.033018112
Epoch:  0015 cost = 8.056357384
Epoch:  

# 2. MNIST MLP로 분류 - 레이어 깊게 쌓아보고, ReLU도 이용해보기!

In [2]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# 3개의 레이어!
linear1 = torch.nn.Linear(784, 256, bias = True).to(device) # 784(28 * 28)를 입력으로 받아 256 출력!
linear2 = torch.nn.Linear(256, 256, bias = True).to(device) # 256 입력 받아 256 출력!
linear3 = torch.nn.Linear(256, 10, bias = True).to(device) # 256 입력 받아 10(0~9) 출력!
relu = torch.nn.ReLU()

# weight 초기화
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

# 모델
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device) # 마지막 레이어 이후 활성화 함수를 넣지 않은 이유는, 밑에서 CrossEntropyLoss()를 이용해 Loss를 계산할 때 그 함수에서 자동으로 softmax 활성화함수 처리를 해주기 때문

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 학습
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = len(data_loader)
  
    for X, Y in data_loader:
        X = X.view(-1, 28*28).to(device)
        Y = Y.to(device)
    
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
    
        optimizer.zero_grad()
        cost.backward()
        avg_cost += cost / total_batch
    
        print("Epoch: ", "%04d" % (epoch+1), "cost =", "{:.9f}".format(avg_cost)) # Accuracy : 94%

Epoch:  0001 cost = 2.483706951
Epoch:  0001 cost = 5.282304764
Epoch:  0001 cost = 7.655402184
Epoch:  0001 cost = 9.732762337
Epoch:  0001 cost = 12.102500916
Epoch:  0001 cost = 14.454638481
Epoch:  0001 cost = 16.815938950
Epoch:  0001 cost = 19.245492935
Epoch:  0001 cost = 21.876106262
Epoch:  0001 cost = 24.319629669
Epoch:  0001 cost = 27.009393692
Epoch:  0001 cost = 29.058816910
Epoch:  0001 cost = 31.346548080
Epoch:  0001 cost = 33.887062073
Epoch:  0001 cost = 36.650547028
Epoch:  0001 cost = 38.996829987
Epoch:  0001 cost = 41.442916870
Epoch:  0001 cost = 44.018062592
Epoch:  0001 cost = 46.283622742
Epoch:  0001 cost = 48.904701233
Epoch:  0001 cost = 51.464134216
Epoch:  0001 cost = 53.895229340
Epoch:  0001 cost = 56.154678345
Epoch:  0001 cost = 58.813972473
Epoch:  0001 cost = 61.322399139
Epoch:  0001 cost = 63.831508636
Epoch:  0001 cost = 66.467529297
Epoch:  0001 cost = 69.119476318
Epoch:  0001 cost = 71.803138733
Epoch:  0001 cost = 74.347236633
Epoch:  0001 c

Epoch:  0001 cost = 617.892333984
Epoch:  0001 cost = 620.414245605
Epoch:  0001 cost = 623.111633301
Epoch:  0001 cost = 625.772216797
Epoch:  0001 cost = 628.236511230
Epoch:  0001 cost = 630.984497070
Epoch:  0001 cost = 633.529296875
Epoch:  0001 cost = 636.167053223
Epoch:  0001 cost = 638.366516113
Epoch:  0001 cost = 641.368225098
Epoch:  0001 cost = 644.040771484
Epoch:  0001 cost = 646.552062988
Epoch:  0001 cost = 648.986755371
Epoch:  0001 cost = 651.474792480
Epoch:  0001 cost = 654.247436523
Epoch:  0001 cost = 656.698425293
Epoch:  0001 cost = 659.193786621
Epoch:  0001 cost = 661.565063477
Epoch:  0001 cost = 664.035522461
Epoch:  0001 cost = 666.686096191
Epoch:  0001 cost = 669.020202637
Epoch:  0001 cost = 671.581298828
Epoch:  0001 cost = 673.806640625
Epoch:  0001 cost = 676.243713379
Epoch:  0001 cost = 679.040588379
Epoch:  0001 cost = 681.614624023
Epoch:  0001 cost = 684.146362305
Epoch:  0001 cost = 686.557739258
Epoch:  0001 cost = 689.086059570
Epoch:  0001 c

Epoch:  0001 cost = 1206.845825195
Epoch:  0001 cost = 1209.128784180
Epoch:  0001 cost = 1211.695678711
Epoch:  0001 cost = 1214.135375977
Epoch:  0001 cost = 1216.401123047
Epoch:  0001 cost = 1218.790283203
Epoch:  0001 cost = 1221.279785156
Epoch:  0001 cost = 1223.759399414
Epoch:  0001 cost = 1226.377929688
Epoch:  0001 cost = 1228.796630859
Epoch:  0001 cost = 1231.289184570
Epoch:  0001 cost = 1233.872558594
Epoch:  0001 cost = 1236.341918945
Epoch:  0001 cost = 1238.791015625
Epoch:  0001 cost = 1241.246215820
Epoch:  0001 cost = 1243.351196289
Epoch:  0001 cost = 1245.796508789
Epoch:  0001 cost = 1248.103637695
Epoch:  0001 cost = 1250.259765625
Epoch:  0001 cost = 1252.546264648
Epoch:  0001 cost = 1254.931396484
Epoch:  0001 cost = 1257.476928711
Epoch:  0001 cost = 1259.716186523
Epoch:  0001 cost = 1262.599975586
Epoch:  0001 cost = 1265.072143555
Epoch:  0001 cost = 1267.418945312
Epoch:  0001 cost = 1270.007080078
Epoch:  0001 cost = 1272.374633789
Epoch:  0001 cost = 

Epoch:  0002 cost = 318.253234863
Epoch:  0002 cost = 320.911163330
Epoch:  0002 cost = 322.926361084
Epoch:  0002 cost = 325.534149170
Epoch:  0002 cost = 328.508728027
Epoch:  0002 cost = 331.381256104
Epoch:  0002 cost = 334.077270508
Epoch:  0002 cost = 336.507324219
Epoch:  0002 cost = 339.106719971
Epoch:  0002 cost = 341.311004639
Epoch:  0002 cost = 343.904510498
Epoch:  0002 cost = 346.038269043
Epoch:  0002 cost = 348.406036377
Epoch:  0002 cost = 350.917175293
Epoch:  0002 cost = 353.424133301
Epoch:  0002 cost = 355.935638428
Epoch:  0002 cost = 358.212890625
Epoch:  0002 cost = 360.697570801
Epoch:  0002 cost = 363.110443115
Epoch:  0002 cost = 365.490142822
Epoch:  0002 cost = 367.913574219
Epoch:  0002 cost = 370.163696289
Epoch:  0002 cost = 372.906616211
Epoch:  0002 cost = 375.603729248
Epoch:  0002 cost = 378.471984863
Epoch:  0002 cost = 381.110015869
Epoch:  0002 cost = 383.662963867
Epoch:  0002 cost = 386.124603271
Epoch:  0002 cost = 389.029815674
Epoch:  0002 c

Epoch:  0002 cost = 921.373718262
Epoch:  0002 cost = 924.197753906
Epoch:  0002 cost = 926.490783691
Epoch:  0002 cost = 928.699829102
Epoch:  0002 cost = 930.995056152
Epoch:  0002 cost = 933.276916504
Epoch:  0002 cost = 935.505615234
Epoch:  0002 cost = 937.983947754
Epoch:  0002 cost = 940.428894043
Epoch:  0002 cost = 942.889282227
Epoch:  0002 cost = 945.769104004
Epoch:  0002 cost = 948.407043457
Epoch:  0002 cost = 950.661499023
Epoch:  0002 cost = 953.064880371
Epoch:  0002 cost = 955.440612793
Epoch:  0002 cost = 958.049987793
Epoch:  0002 cost = 960.748718262
Epoch:  0002 cost = 963.644104004
Epoch:  0002 cost = 966.300354004
Epoch:  0002 cost = 968.626159668
Epoch:  0002 cost = 971.076171875
Epoch:  0002 cost = 973.636230469
Epoch:  0002 cost = 976.441772461
Epoch:  0002 cost = 979.050231934
Epoch:  0002 cost = 981.467773438
Epoch:  0002 cost = 983.768676758
Epoch:  0002 cost = 986.340698242
Epoch:  0002 cost = 989.208923340
Epoch:  0002 cost = 991.663635254
Epoch:  0002 c

...

Epoch:  0013 cost = 1432.261840820
Epoch:  0013 cost = 1434.835449219
Epoch:  0013 cost = 1437.055541992
Epoch:  0013 cost = 1439.455200195
Epoch:  0013 cost = 1441.754394531
Epoch:  0013 cost = 1444.064453125
Epoch:  0013 cost = 1446.369262695
Epoch:  0013 cost = 1448.809204102
Epoch:  0013 cost = 1450.985351562
Epoch:  0013 cost = 1453.226562500
Epoch:  0013 cost = 1455.701660156
Epoch:  0013 cost = 1457.963989258
Epoch:  0013 cost = 1460.369384766
Epoch:  0013 cost = 1462.872192383
Epoch:  0013 cost = 1465.075073242
Epoch:  0013 cost = 1467.022705078
Epoch:  0013 cost = 1469.645751953
Epoch:  0013 cost = 1472.015014648
Epoch:  0013 cost = 1474.752319336
Epoch:  0014 cost = 2.681282520
Epoch:  0014 cost = 5.276569366
Epoch:  0014 cost = 7.789570808
Epoch:  0014 cost = 10.262197495
Epoch:  0014 cost = 12.790023804
Epoch:  0014 cost = 15.411021233
Epoch:  0014 cost = 18.085027695
Epoch:  0014 cost = 20.432928085
Epoch:  0014 cost = 22.683469772
Epoch:  0014 cost = 25.174345016
Epoch:  

Epoch:  0014 cost = 556.742492676
Epoch:  0014 cost = 558.944580078
Epoch:  0014 cost = 561.369140625
Epoch:  0014 cost = 563.975952148
Epoch:  0014 cost = 566.390625000
Epoch:  0014 cost = 568.584899902
Epoch:  0014 cost = 571.214782715
Epoch:  0014 cost = 573.311950684
Epoch:  0014 cost = 575.879699707
Epoch:  0014 cost = 577.875671387
Epoch:  0014 cost = 580.606201172
Epoch:  0014 cost = 583.058410645
Epoch:  0014 cost = 585.532653809
Epoch:  0014 cost = 588.250427246
Epoch:  0014 cost = 590.348999023
Epoch:  0014 cost = 592.963134766
Epoch:  0014 cost = 595.195068359
Epoch:  0014 cost = 597.668762207
Epoch:  0014 cost = 599.933227539
Epoch:  0014 cost = 602.164184570
Epoch:  0014 cost = 604.838684082
Epoch:  0014 cost = 607.041625977
Epoch:  0014 cost = 609.513061523
Epoch:  0014 cost = 611.894042969
Epoch:  0014 cost = 614.406738281
Epoch:  0014 cost = 616.883056641
Epoch:  0014 cost = 619.528381348
Epoch:  0014 cost = 622.071960449
Epoch:  0014 cost = 624.448181152
Epoch:  0014 c

Epoch:  0014 cost = 1178.511962891
Epoch:  0014 cost = 1180.791992188
Epoch:  0014 cost = 1183.353271484
Epoch:  0014 cost = 1185.923095703
Epoch:  0014 cost = 1188.153930664
Epoch:  0014 cost = 1190.565063477
Epoch:  0014 cost = 1192.933105469
Epoch:  0014 cost = 1195.308715820
Epoch:  0014 cost = 1197.388305664
Epoch:  0014 cost = 1199.770141602
Epoch:  0014 cost = 1202.271362305
Epoch:  0014 cost = 1204.961547852
Epoch:  0014 cost = 1207.815917969
Epoch:  0014 cost = 1210.163940430
Epoch:  0014 cost = 1212.359008789
Epoch:  0014 cost = 1214.869750977
Epoch:  0014 cost = 1216.895996094
Epoch:  0014 cost = 1219.199340820
Epoch:  0014 cost = 1221.819702148
Epoch:  0014 cost = 1224.347778320
Epoch:  0014 cost = 1226.747192383
Epoch:  0014 cost = 1229.120605469
Epoch:  0014 cost = 1231.668457031
Epoch:  0014 cost = 1234.304809570
Epoch:  0014 cost = 1236.745117188
Epoch:  0014 cost = 1239.168701172
Epoch:  0014 cost = 1241.448852539
Epoch:  0014 cost = 1243.901733398
Epoch:  0014 cost = 

Epoch:  0015 cost = 309.247222900
Epoch:  0015 cost = 311.674499512
Epoch:  0015 cost = 314.042449951
Epoch:  0015 cost = 316.372955322
Epoch:  0015 cost = 319.364227295
Epoch:  0015 cost = 321.940948486
Epoch:  0015 cost = 324.209625244
Epoch:  0015 cost = 326.975311279
Epoch:  0015 cost = 329.646606445
Epoch:  0015 cost = 331.795104980
Epoch:  0015 cost = 334.396881104
Epoch:  0015 cost = 336.958496094
Epoch:  0015 cost = 339.221862793
Epoch:  0015 cost = 341.631225586
Epoch:  0015 cost = 344.152648926
Epoch:  0015 cost = 346.458923340
Epoch:  0015 cost = 348.917694092
Epoch:  0015 cost = 351.186096191
Epoch:  0015 cost = 353.514984131
Epoch:  0015 cost = 356.355712891
Epoch:  0015 cost = 358.865814209
Epoch:  0015 cost = 361.265289307
Epoch:  0015 cost = 363.898162842
Epoch:  0015 cost = 366.535095215
Epoch:  0015 cost = 368.911560059
Epoch:  0015 cost = 371.355804443
Epoch:  0015 cost = 373.752349854
Epoch:  0015 cost = 376.371978760
Epoch:  0015 cost = 379.258666992
Epoch:  0015 c

Epoch:  0015 cost = 907.446533203
Epoch:  0015 cost = 909.768249512
Epoch:  0015 cost = 912.343139648
Epoch:  0015 cost = 914.587646484
Epoch:  0015 cost = 916.880554199
Epoch:  0015 cost = 919.401916504
Epoch:  0015 cost = 921.815917969
Epoch:  0015 cost = 924.330200195
Epoch:  0015 cost = 926.560668945
Epoch:  0015 cost = 929.042358398
Epoch:  0015 cost = 932.013061523
Epoch:  0015 cost = 934.382019043
Epoch:  0015 cost = 937.009094238
Epoch:  0015 cost = 939.665344238
Epoch:  0015 cost = 941.944335938
Epoch:  0015 cost = 944.639831543
Epoch:  0015 cost = 946.842834473
Epoch:  0015 cost = 949.359497070
Epoch:  0015 cost = 951.699951172
Epoch:  0015 cost = 954.045532227
Epoch:  0015 cost = 956.304504395
Epoch:  0015 cost = 958.581054688
Epoch:  0015 cost = 961.000976562
Epoch:  0015 cost = 963.588256836
Epoch:  0015 cost = 966.009765625
Epoch:  0015 cost = 968.309570312
Epoch:  0015 cost = 971.093750000
Epoch:  0015 cost = 973.492492676
Epoch:  0015 cost = 975.847778320
Epoch:  0015 c